In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  
from keras.layers import Dropout

In [3]:
data = pd.read_csv('modeling/Data/Data Given/MMM.csv')

In [4]:
d2 = data.dropna()
d2.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1/2/2009,57.549999,59.389999,57.520000,59.189999,45.690918,5313900.0
1,1/5/2009,58.790001,59.090000,58.110001,58.500000,45.158264,3768800.0
2,1/6/2009,58.740002,59.810001,58.610001,59.189999,45.690918,4966200.0
3,1/7/2009,58.610001,58.959999,57.580002,58.070000,44.826347,4598100.0
4,1/8/2009,57.680000,58.709999,57.450001,58.580002,45.220032,3944900.0


In [6]:
mm = d2.iloc[:,4:5].values

In [7]:
scaler = MinMaxScaler(feature_range = (0, 1))

trans_mm= scaler.fit_transform(mm)

In [8]:
features_set = []  
labels = []  
for i in range(60, len(mm)):  
    features_set.append(trans_mm[i-60:i, 0])
    labels.append(trans_mm[i, 0])
features_set, labels = np.array(features_set), np.array(labels)
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))

In [ ]:
TRAIN_SIZE = 0.60

train_size = int(len(trans_mm) * TRAIN_SIZE)
test_size = len(trans_mm) - train_size
train, test = trans_mm[0:train_size, :], trans_mm[train_size:len(trans_mm), :]

def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(trans_mm) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(trans_mm[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))
window_size = 1
train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)

train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))


In [ ]:
def fit_model(train_X, train_Y, window_size = 1):
    model = Sequential()
    
    model.add(LSTM(4, 
                   input_shape = (1, window_size)))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    model.fit(train_X, 
              train_Y, 
              epochs = 100, 
              batch_size = 1, 
              verbose = 2)
    
    return(model)

# Fit the first model.
model1 = fit_model(train_X, train_Y, window_size)

In [ ]:
def predict_and_score(model, X, Y):
    # Make predictions on the original scale of the data.
    pred = scaler.inverse_transform(model.predict(X))
    # Prepare Y data to also be on the original scale for interpretability.
    orig_data = scaler.inverse_transform([Y])
    # Calculate RMSE.
    score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    return(score, pred)

rmse_train, train_predict = predict_and_score(model1, train_X, train_Y)
rmse_test, test_predict = predict_and_score(model1, test_X, test_Y)

In [ ]:
# Start with training predictions.
train_predict_plot = np.empty_like(trans_mm)
train_predict_plot[:, :] = np.nan
train_predict_plot[window_size:len(train_predict) + window_size, :] = train_predict

# Add test predictions.
test_predict_plot = np.empty_like(trans_mm)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict) + (window_size * 2) + 1:len(trans_mm) - 1, :] = test_predict

# Create the plot.
plt.figure(figsize = (15, 5))
plt.plot(scaler.inverse_transform(trans_mm), label = "True value")
plt.plot(train_predict_plot, label = "Training set prediction")
plt.plot(test_predict_plot, label = "Test set prediction")

In [9]:
test_inputs= d2.iloc[:,1:2].values

In [10]:
test_inputs = test_inputs.reshape(-1,1)  
test_inputs = scaler.transform(test_inputs) 
test_features = []  
for i in range(60, len(test_inputs)):  
    test_features.append(test_inputs[i-60:i, 0])
test_features = np.array(test_features)  
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1)) 

In [12]:
features_set.shape,test_features.shape

((2456, 60, 1), (2456, 60, 1))

In [13]:
model = Sequential() 
model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))
model.add(Dropout(0.2)) 
model.add(LSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))

model.add(LSTM(units=50))  
model.add(Dropout(0.2))
model.add(Dense(units = 1))  
model.compile(optimizer = 'adam', loss = 'mean_squared_error') 

In [14]:
model.fit(features_set, labels, epochs = 100, batch_size = 32)

Epoch 1/100
2456/2456 [==============================] - 21s 9ms/step - loss: 0.0176
Epoch 2/100
2456/2456 [==============================] - 19s 8ms/step - loss: 0.0027
Epoch 3/100
2456/2456 [==============================] - 18s 7ms/step - loss: 0.0028
Epoch 4/100
2456/2456 [==============================] - 18s 7ms/step - loss: 0.0024
Epoch 5/100
2456/2456 [==============================] - 19s 8ms/step - loss: 0.0022
Epoch 6/100
2456/2456 [==============================] - 19s 8ms/step - loss: 0.0022
Epoch 7/100
2456/2456 [==============================] - 18s 7ms/step - loss: 0.0021
Epoch 8/100
2456/2456 [==============================] - 18s 7ms/step - loss: 0.0022
Epoch 9/100
2456/2456 [==============================] - 19s 8ms/step - loss: 0.0017
Epoch 10/100
2456/2456 [==============================] - 19s 8ms/step - loss: 0.0018
Epoch 11/100
2456/2456 [==============================] - 19s 8ms/step - loss: 0.0019
Epoch 12/100
2456/2456 [==============================] - 19s 8

2456/2456 [==============================] - 18s 7ms/step - loss: 5.7211e-04
Epoch 94/100
2456/2456 [==============================] - 19s 8ms/step - loss: 5.6146e-04
Epoch 95/100
2456/2456 [==============================] - 19s 8ms/step - loss: 6.0059e-04
Epoch 96/100
2456/2456 [==============================] - 19s 8ms/step - loss: 6.5816e-04
Epoch 97/100
2456/2456 [==============================] - 18s 7ms/step - loss: 6.5374e-04
Epoch 98/100
2456/2456 [==============================] - 18s 7ms/step - loss: 5.8760e-04
Epoch 99/100
2456/2456 [==============================] - 18s 7ms/step - loss: 6.4308e-04
Epoch 100/100
2456/2456 [==============================] - 19s 8ms/step - loss: 6.1874e-04


In [15]:
predictions = model.predict(test_features)
predictions = scaler.inverse_transform(predictions) 

In [ ]:
plt.figure(figsize=(10,6))  
plt.plot(mm, color='blue', label='Actual')  
plt.plot(predictions , color='red', label='Predicted')    
plt.legend()  
plt.show()